In [16]:
# data processing
import pandas as pd

import psycopg2

# append path to sys
import sys
sys.path.append("..")

# import environment variables
from dotenv import load_dotenv

load_dotenv()

database_credentials = {
    "database": "chimichanga",
    "username": "username",
    "password": "admin",
    "host":"localhost",
    "port":"5432"
}

from src.support.support_creacion_tablas import dropear_tablas
from src.support.database_connection_support import connect_to_database, connect_and_query
import src.support.support_creacion_tablas as sct


Conectar a la base de datos [falta completar]

In [10]:
def establecer_conn(database_name, postgres_pass, usuario, host="localhost", autocommit=True):
    """
    Establece una conexión a una base de datos de PostgreSQL.

    Params:
        - database_name (str): El nombre de la base de datos a la que conectarse.
        - postgres_pass (str): La contraseña del usuario de PostgreSQL.
        - usuario (str): El nombre del usuario de PostgreSQL.
        - host (str, opcional): La dirección del servidor PostgreSQL. Por defecto es "localhost".

    Returns:
        psycopg2.extensions.connection: La conexión establecida a la base de datos PostgreSQL.

    """

    # Crear la conexión a la base de datos PostgreSQL
    conn = psycopg2.connect(
        host=host,
        user=usuario,
        password=postgres_pass,
        database=database_name
    )

    # Establecer la conexión en modo autocommit
    conn.autocommit = autocommit # No hace necesario el uso del commit al final de cada sentencia de insert, delete, etc.
    
    return conn



# conexion a postgres
conn = establecer_conn("postgres", "admin", "postgres")

In [11]:
# conn = connect_to_database(database_credentials)
cur = conn.cursor()
cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", ("sanidad42",)) 

In [12]:
bbdd_existe = cur.fetchone() # recordamos que fetchone nos devuelve unico valor  
type(bbdd_existe)

NoneType

In [15]:
# Si bbdd_existe es None, creamos la base de datos
if not bbdd_existe:
    cur.execute(f"CREATE DATABASE sanidad42;")
    print(f"Base de datos sanidad42 creada con éxito")
else:
    print(f"La base de datos ya existe")
    

Base de datos sanidad42 creada con éxito


In [ ]:
def crear_db(database_name):
    # conexion a postgres
    conn = establecer_conn("postgres", "admin", "my_user") # Nos conectamos a la base de datos de postgres por defecto para poder crear la nueva base de datos
    
    # creamos un cursor con la conexion que acabamos de crear
    cur = conn.cursor()
    
    cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (database_name,))
    
    # Almacenamos en una variable el resultado del fetchone. Si existe tendrá una fila sino será None
    bbdd_existe = cur.fetchone()
    
    # Si bbdd_existe es None, creamos la base de datos
    if not bbdd_existe:
        cur.execute(f"CREATE DATABASE {database_name};")
        print(f"Base de datos {database_name} creada con éxito")
    else:
        print(f"La base de datos ya existe")
        
    # Cerramos el cursor y la conexion
    cur.close()
    conn.close()

Crear tablas

In [34]:
# establecemos la conexión a base de datos creada en el paso anterior
conexion = establecer_conn("sanidad42", "admin", "postgres")

# creamos cada una de las tablas de la base de datos.
conexion.cursor().execute(sct.query_creation_hospitales)
conexion.cursor().execute(sct.query_creation_tipo_hosp)
conexion.cursor().execute(sct.query_creation_gastos)
conexion.cursor().execute(sct.query_creation_ingresos)

In [35]:
cur = conexion.cursor()

## Insertar registros en tablas

In [22]:
ingresos = pd.read_csv("../datos/tablas/ingresos_modificados.csv").iloc[:,2:]
ingresos.head()

,particulares,aseguradoras,aseguradoras_enfermedad,aseguradoreas_trafico,mutuas,tipo,año,ncodi
0,0.0,103677.0,64383.0,39294.0,61456.0,0,2013,"990,00"
1,324687.0,478280.0,392084.0,86196.0,484228.0,0,2013,"680,00"
2,793512.0,1803352.0,1803352.0,0.0,20402.0,0,2013,"411,00"
3,0.0,0.0,0.0,0.0,0.0,0,2013,"241,00"
4,24406.0,63557.0,55091.0,8466.0,44106.0,0,2013,"558,00"


In [46]:
tipo_hospitalizacion = pd.read_csv("../datos/tablas/tabla_tipo_hospitalizacion.csv").iloc[:,1:]
tipo_hospitalizacion.head()

,0,id
0,Hospital,0
1,consulExter,1
2,CMA,2
3,hospDia,3
4,Urgencia,4


In [51]:
tabla_centros = pd.read_csv("../datos/tablas/tabla_centros.csv").iloc[:,1:]
tabla_centros.head()

,id,nombre
0,99000,Hospital_99000
1,68000,Hospital_68000
2,41100,Hospital_41100
3,24100,Hospital_24100
4,55800,Hospital_55800


In [59]:
tabla_centros.drop_duplicates(inplace=True)

In [63]:
gastos = pd.read_csv("../datos/DatosFinales/gastos.csv")
gastos.head()

,año,ncodi,totalcompra,producfarma,materialsani,implantes,restomateriasani,servcontratado,trabajocontratado,xrestocompras,...,xrestoserviexter,gastopersonal,sueldos,indemnizacion,segsocempresa,otrgassocial,dotaamortizacion,perdidadeterioro,xrestogasto,totcompragasto
0,2020,113.0,1535894,344330.0,169443,0.0,169443.0,1877.0,678201.0,342043.0,...,506018.0,26209836,19875799,349499.0,5984538,0.0,298707.0,-6504.0,50165.0,28810972
1,2020,619.0,7884157,971762.0,1749006,493444.0,1255562.0,3800207.0,1248674.0,114508.0,...,2635675.0,7083969,5203283,151380.0,1591074,138232.0,2423752.0,0.0,312984.0,20573925
2,2020,163.0,5453290,126493.0,1412366,574127.0,838239.0,3217684.0,647852.0,48895.0,...,882274.0,2890388,2180177,-402.0,663398,47215.0,202688.0,-54028.0,232471.0,9677715
3,2020,131.0,245802,72318.0,52506,0.0,52506.0,1114.0,24303.0,95561.0,...,487053.0,1887375,1437421,9506.0,416398,24050.0,12084.0,240.0,132044.0,2844476
4,2020,375.0,762218,114702.0,106844,60.0,106784.0,14526.0,188463.0,337683.0,...,192132.0,7764302,5838470,0.0,1837175,88657.0,184413.0,93708.0,1552.0,9037657


### centro hospitalario

```sql
create table if not exists hospitales (
    ncodi INT primary key,
    name VARCHAR(300)
);
```

In [60]:
lista_tuplas_centros = [tuple(row) for _,row in tabla_centros.iterrows()]
lista_tuplas_centros

[(99000, 'Hospital_99000'),
 (68000, 'Hospital_68000'),
 (41100, 'Hospital_41100'),
 (24100, 'Hospital_24100'),
 (55800, 'Hospital_55800'),
 (16700, 'Hospital_16700'),
 (24800, 'Hospital_24800'),
 (22400, 'Hospital_22400'),
 (59100, 'Hospital_59100'),
 (58600, 'Hospital_58600'),
 (45300, 'Hospital_45300'),
 (11300, 'Hospital_11300'),
 (61900, 'Hospital_61900'),
 (16300, 'Hospital_16300'),
 (13100, 'Hospital_13100'),
 (37500, 'Hospital_37500'),
 (109000, 'Hospital_109000'),
 (76400, 'Hospital_76400'),
 (1400, 'Hospital_1400'),
 (65700, 'Hospital_65700'),
 (36200, 'Hospital_36200'),
 (98900, 'Hospital_98900'),
 (14500, 'Hospital_14500'),
 (51100, 'Hospital_51100'),
 (53000, 'Hospital_53000'),
 (600, 'Hospital_600'),
 (16100, 'Hospital_16100'),
 (62300, 'Hospital_62300'),
 (48900, 'Hospital_48900'),
 (87500, 'Hospital_87500'),
 (94000, 'Hospital_94000'),
 (94600, 'Hospital_94600'),
 (102300, 'Hospital_102300'),
 (105400, 'Hospital_105400'),
 (105700, 'Hospital_105700'),
 (103100, 'Hospita

In [61]:
cur.executemany(
"INSERT INTO hospitales (ncodi,name) VALUES (%s,%s)",lista_tuplas_centros
)

### Tipo hospital

```sql
create table if not exists tipo_hospitalizacion (
    tipo_id primary key,
    nombre VARCHAR(100) unique not null
    );
```

In [49]:
lista_tuplas_tipo_hospitalizacion = [tuple([row[1],row[0]]) for _,row in tipo_hospitalizacion.iterrows()]
lista_tuplas_tipo_hospitalizacion

C:\Users\Miguel\AppData\Local\Temp\ipykernel_43236\3373371235.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lista_tuplas_tipo_hospitalizacion = [tuple([row[1],row[0]]) for _,row in tipo_hospitalizacion.iterrows()]


[(0, 'Hospital'),
 (1, 'consulExter'),
 (2, 'CMA'),
 (3, 'hospDia'),
 (4, 'Urgencia'),
 (5, 'hospDom')]

In [50]:
cur.executemany(
"INSERT INTO tipo_hospitalizacion (tipo_id,nombre) VALUES (%s,%s)",
lista_tuplas_tipo_hospitalizacion
)

### gastos

In [66]:
lista_tupla_gastos = [tuple(row) for _,row in gastos.iterrows()]
lista_tupla_gastos

[(2020.0,
  113.0,
  1535894.0,
  344330.0,
  169443.0,
  0.0,
  169443.0,
  1877.0,
  678201.0,
  342043.0,
  3039.0,
  719835.0,
  213817.0,
  506018.0,
  26209836.0,
  19875799.0,
  349499.0,
  5984538.0,
  0.0,
  298707.0,
  -6504.0,
  50165.0,
  28810972.0),
 (2020.0,
  619.0,
  7884157.0,
  971762.0,
  1749006.0,
  493444.0,
  1255562.0,
  3800207.0,
  1248674.0,
  114508.0,
  24083.0,
  2844980.0,
  209305.0,
  2635675.0,
  7083969.0,
  5203283.0,
  151380.0,
  1591074.0,
  138232.0,
  2423752.0,
  0.0,
  312984.0,
  20573925.0),
 (2020.0,
  163.0,
  5453290.0,
  126493.0,
  1412366.0,
  574127.0,
  838239.0,
  3217684.0,
  647852.0,
  48895.0,
  -26827.0,
  979733.0,
  97459.0,
  882274.0,
  2890388.0,
  2180177.0,
  -402.0,
  663398.0,
  47215.0,
  202688.0,
  -54028.0,
  232471.0,
  9677715.0),
 (2020.0,
  131.0,
  245802.0,
  72318.0,
  52506.0,
  0.0,
  52506.0,
  1114.0,
  24303.0,
  95561.0,
  0.0,
  566931.0,
  79878.0,
  487053.0,
  1887375.0,
  1437421.0,
  9506.0,
  4

```sql
create table if not exists gastos (
    gastos_id INT primary key,
    año INT not null,
    ncodi INT not null,
    totalcompra NUMERIC, 
    producfarma NUMERIC, 
    materialsani NUMERIC, 
    implantes NUMERIC, 
    restomateriasani NUMERIC, 
    servcontratado NUMERIC, 
    trabajocontratado NUMERIC, 
    xrestocompras NUMERIC, 
    variaexistencias NUMERIC, 
    servexteriores NUMERIC, 
    sumistro NUMERIC, 
    xrestoserviexter NUMERIC, 
    gastopersonal NUMERIC, 
    sueldos NUMERIC, 
    indemnizacion NUMERIC, 
    segsocempresa NUMERIC, 
    otrgassocial NUMERIC, 
    dotaamortizacion NUMERIC, 
    perdidadeterioro NUMERIC, 
    xrestogasto NUMERIC, 
    totcompragasto NUMERIC,
    foreign key (ncodi) references hospitales(ncodi)
);
```

In [73]:
cur.executemany(
"INSERT INTO gastos VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (gastos_id) DO NOTHING",
lista_tupla_gastos
)

ForeignKeyViolation: inserción o actualización en la tabla «gastos» viola la llave foránea «gastos_ncodi_fkey»
DETAIL:  La llave (ncodi)=(1535894) no está presente en la tabla «hospitales».
